In [1]:
import numpy as np

In [82]:
def parse_input(filepath):
    with open(filepath, 'r') as f:
        nums = [[int(x) for x in l.strip()] for l in f]
    a = np.array(nums, dtype=np.uint8)

    return a

In [83]:
a = parse_input('test_input.txt')

In [84]:
a

array([[5, 4, 8, 3, 1, 4, 3, 2, 2, 3],
       [2, 7, 4, 5, 8, 5, 4, 7, 1, 1],
       [5, 2, 6, 4, 5, 5, 6, 1, 7, 3],
       [6, 1, 4, 1, 3, 3, 6, 1, 4, 6],
       [6, 3, 5, 7, 3, 8, 5, 4, 7, 8],
       [4, 1, 6, 7, 5, 2, 4, 6, 4, 5],
       [2, 1, 7, 6, 8, 4, 1, 7, 2, 1],
       [6, 8, 8, 2, 8, 8, 1, 1, 3, 4],
       [4, 8, 4, 6, 8, 4, 8, 5, 5, 4],
       [5, 2, 8, 3, 7, 5, 1, 5, 2, 6]], dtype=uint8)

In [85]:
energies = a

In [86]:
def num_neighbours_flashed(flashed):
    out =  (
        flashed[0:-2, 1:-1] +
        flashed[2:, 1:-1] +
        flashed[1:-1, 0:-2] +
        flashed[1:-1, 2:] +
        flashed[0:-2, 0:-2] +
        flashed[0:-2, 2:] +
        flashed[2:, 0:-2] +
        flashed[2:, 2:]
    )

    return out

In [106]:
def model_step(energies):
    flashes = np.zeros(energies.shape, dtype=np.uint8)
    flashed = np.pad(energies, 1, constant_values=0)

    energies += 1
    flashed[1:-1, 1:-1] = (energies > 9)
    flashes += flashed[1:-1, 1:-1]
    
    while np.any(flashed):
        neighbours_flashed = num_neighbours_flashed(flashed)
        energies += neighbours_flashed
        flashed[1:-1, 1:-1] = ((energies > 9) & (flashes == 0))
        flashes += flashed[1:-1, 1:-1]

    np.place(energies, (flashes > 0), 0)

    return flashes

In [120]:
def octopus_model(energies, steps):
    work_energies = energies.copy()
    num_flashes = np.zeros(energies.shape, dtype=np.uint8)
    
    for i in range(steps):
        num_flashes += model_step(work_energies)

    return work_energies, num_flashes

In [121]:
from itertools import count

In [123]:
def first_synchronized_flash(energies):
    work_energies = energies

    for i in count(1):
        if (np.all(model_step(work_energies) > 0)):
            return i

In [125]:
first_synchronized_flash(energies)

195

In [124]:
energies

array([[5, 4, 8, 3, 1, 4, 3, 2, 2, 3],
       [2, 7, 4, 5, 8, 5, 4, 7, 1, 1],
       [5, 2, 6, 4, 5, 5, 6, 1, 7, 3],
       [6, 1, 4, 1, 3, 3, 6, 1, 4, 6],
       [6, 3, 5, 7, 3, 8, 5, 4, 7, 8],
       [4, 1, 6, 7, 5, 2, 4, 6, 4, 5],
       [2, 1, 7, 6, 8, 4, 1, 7, 2, 1],
       [6, 8, 8, 2, 8, 8, 1, 1, 3, 4],
       [4, 8, 4, 6, 8, 4, 8, 5, 5, 4],
       [5, 2, 8, 3, 7, 5, 1, 5, 2, 6]], dtype=uint8)

In [112]:
out_energies, flashes = octopus_model(energies, 100)

In [113]:
out_energies

array([[0, 3, 9, 7, 6, 6, 6, 8, 6, 6],
       [0, 7, 4, 9, 7, 6, 6, 9, 1, 8],
       [0, 0, 5, 3, 9, 7, 6, 9, 3, 3],
       [0, 0, 0, 4, 2, 9, 7, 8, 2, 2],
       [0, 0, 0, 4, 2, 2, 9, 8, 9, 2],
       [0, 0, 5, 3, 2, 2, 2, 8, 7, 7],
       [0, 5, 3, 2, 2, 2, 2, 9, 6, 6],
       [9, 3, 2, 2, 2, 2, 8, 9, 6, 6],
       [7, 9, 2, 2, 2, 8, 6, 8, 6, 6],
       [6, 7, 8, 9, 9, 9, 8, 7, 6, 6]], dtype=uint8)

In [114]:
flashes

array([[15, 17, 17, 16, 16, 16, 16, 16, 15, 14],
       [17, 19, 17, 16, 17, 16, 16, 17, 20, 15],
       [17, 17, 17, 17, 16, 16, 16, 16, 18, 17],
       [17, 17, 18, 17, 17, 16, 16, 16, 18, 17],
       [17, 17, 18, 22, 17, 18, 16, 16, 17, 17],
       [17, 17, 18, 17, 17, 17, 17, 16, 16, 16],
       [17, 17, 17, 17, 17, 17, 17, 16, 16, 16],
       [15, 17, 17, 17, 17, 17, 17, 19, 17, 15],
       [14, 15, 17, 17, 17, 17, 18, 18, 15, 15],
       [14, 14, 15, 16, 16, 16, 16, 15, 15, 14]], dtype=uint8)

In [115]:
flashes.sum()

1656

In [49]:
flashes = model_step(energies)

In [96]:
test = parse_input('small_test_input.txt')

In [97]:
test

array([[1, 1, 1, 1, 1],
       [1, 9, 9, 9, 1],
       [1, 9, 1, 9, 1],
       [1, 9, 9, 9, 1],
       [1, 1, 1, 1, 1]], dtype=uint8)

In [98]:
out_test, test_flashes = octopus_model(test, 2)

In [99]:
out_test

array([[4, 5, 6, 5, 4],
       [5, 1, 1, 1, 5],
       [6, 1, 1, 1, 6],
       [5, 1, 1, 1, 5],
       [4, 5, 6, 5, 4]], dtype=uint8)

In [100]:
test_flashes

array([[0, 0, 0, 0, 0],
       [0, 1, 1, 1, 0],
       [0, 1, 1, 1, 0],
       [0, 1, 1, 1, 0],
       [0, 0, 0, 0, 0]], dtype=uint8)

In [116]:
actual = parse_input('input.txt')

In [117]:
actual

array([[2, 4, 7, 8, 6, 6, 8, 3, 2, 4],
       [4, 2, 8, 3, 4, 7, 4, 1, 2, 5],
       [1, 6, 6, 3, 4, 6, 3, 3, 7, 4],
       [1, 7, 3, 8, 2, 7, 1, 3, 2, 3],
       [4, 2, 8, 5, 7, 4, 4, 8, 6, 1],
       [3, 5, 5, 1, 3, 1, 1, 5, 1, 5],
       [8, 5, 7, 4, 3, 3, 5, 4, 3, 8],
       [7, 8, 4, 3, 5, 2, 5, 8, 2, 6],
       [1, 3, 6, 6, 2, 3, 7, 5, 7, 7],
       [3, 5, 5, 4, 6, 8, 7, 2, 2, 6]], dtype=uint8)

In [119]:
(octopus_model(actual, 100)[1]).sum()

1700

In [126]:
first_synchronized_flash(actual)

273